In [ ]:
import logging
from logging import DEBUG

import pandas as pd
from influxdb_client import InfluxDBClient

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
client = InfluxDBClient(url="https://influx.konzept.space/",
                        # this is a public readonly token, so while not best practice, there's no danger in hard-coding it here :)
                        token="0yLbh-D7RMe1sX1iIudFel8CcqCI8sVfuRTaliUp56MgE6kub8-nSd05_EJ4zTTKt0lUzw8zcO73zL9QhC3jtA==",
                        org="api.existenz.ch",
                        debug=False,
                        timeout=60_000)

Fetch hourly mean for discharge (rate of flow) and temperature for Thun (id 2030) and Bern, Schönau (id 2135) of the last few days.

In [ ]:
df = client.query_api().query_data_frame(
    """
from(bucket: "existenzApi")
  |> range(start: -5d)
  |> filter(fn: (r) => r["_measurement"] == "hydro")
  |> filter(fn: (r) => r["_field"] == "temperature" or r["_field"] == "flow")
  |> filter(fn: (r) => r["loc"] == "2135" or r["loc"] == "2030")
  |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> drop(columns: ["_start", "result", "_stop", "table", "_measurement"])
    """
)

df = df[['_time', 'loc', 'flow', 'temperature']]
df

In [ ]:
df = client.query_api().query_data_frame(
    """
from(bucket: "existenzApi")
  |> range(start: -5d)
  |> filter(fn: (r) => r["_measurement"] == "hydro")
  |> filter(fn: (r) => r["_field"] == "temperature")
  |> filter(fn: (r) => r["loc"] == "2135" or r["loc"] == "2030")
  |> aggregateWindow(every: 1h, fn: mean, createEmpty: false)
  |> pivot(rowKey: ["_time"], columnKey: ["_field"], valueColumn: "_value")
  |> drop(columns: ["_start", "result", "_stop", "table", "_measurement"])
    """
)

df = df[['_time', 'loc', 'temperature']]
df

In [ ]:
import logging
from logging import DEBUG
logging.basicConfig(level=DEBUG)

In [ ]:
from aare.constants import LOC_BERN, LOC_THUN
from aare.remote_influx_store import RemoteExistenzStore

store = RemoteExistenzStore()

In [ ]:
store.query_hydro("2020-01-01T00:00:00Z", LOC_BERN)

In [ ]:
df.dtypes

In [ ]:
df.pivot(index='_time', columns='loc', values='flow').plot()

In [ ]:
df.pivot(index='_time', columns='loc', values='temperature').plot()

In [ ]:
client.query_api().query_data_frame(
    """
from(bucket: "existenzApi")
  |> range(start: -10000d)
  |> filter(fn: (r) => r["_measurement"] == "hydro")
  |> filter(fn: (r) => r["_field"] == "temperature" or r["_field"] == "flow")
  |> filter(fn: (r) => r["loc"] == "2135" or r["loc"] == "2030")
  |> first()
     |> pivot(rowKey:["_time"], columnKey: ["_field"], valueColumn: "_value")
    """
)


WARNING: long-running query

Check how many (aggregated) data points there are for the two most interesting locations on the entire dataset.
Without data loss, each year would have 8760 data points (365 days * 24 hours).

In [ ]:
n_measurements_per_year = client.query_api().query_data_frame(
    """
import "date"

from(bucket: "existenzApi")
  |> range(start: -25y)
  |> filter(fn: (r) => r["_measurement"] == "hydro")
  |> filter(fn: (r) => r["_field"] == "temperature" or r["_field"] == "flow")
  |> filter(fn: (r) => r["loc"] == "2135" or r["loc"] == "2030")
  |> aggregateWindow(every: 1h, fn: first, createEmpty: false)
  |> map(fn: (r) => ({ r with year: date.truncate(t: r._time, unit: 1y) }))
  |> group(columns: ["year", "_field", "loc"])
  |> count()
  // |> pivot(rowKey:["year"], columnKey: ["_field"], valueColumn: "_value")
    """
)

n_measurements_per_year